In [1]:
from collections import Counter, defaultdict

import cv2
import matplotlib.pyplot as plt
import numpy as np

# from .exceptions import InvalidDetectionData, MissingCalibrationData
# from .intrinsic import Camera

In [ ]:
# def get_projection_matrices(cameras: list[Camera]):
#     proj_matrices = []
#     for cam in cameras:
#         K = cam.intrinsic
#         extrinsic = cam.extrinsic_matrix

#         R = extrinsic[:3, :3]
#         t = extrinsic[:3, 3].reshape(3, 1)

#         Rt = np.hstack((R, t))  # shape (3, 4)
#         P = K @ Rt  # projection matrix: shape (3, 4)
#         proj_matrices.append(P)
#     return proj_matrices


# def undistort_corners(corners, camera: Camera):
#     undistorted_corners = cv2.undistortPoints(
#         corners.reshape(-1, 1, 2),
#         cameraMatrix=camera.intrinsic,
#         distCoeffs=camera.distortion_coeff,
#         P=camera.intrinsic,  # reprojects into pixel coords
#     ).reshape(-1, 2)
#     return undistorted_corners


# def triangulate_n_views(image_points, projection_matrices):
#     A = []
#     for x, P in zip(image_points, projection_matrices):
#         x, y = x[0], x[1]
#         A.append(x * P[2] - P[0])
#         A.append(y * P[2] - P[1])
#     A = np.stack(A)
#     _, _, Vt = np.linalg.svd(A)
#     X = Vt[-1]
#     X = X / X[3]
#     return X[:3]


# def reproject(P, X):
#     X_h = np.append(X, 1.0)
#     proj = P @ X_h
#     return proj[:2] / proj[2]


# def check_accuracy(
#     detections,
#     cameras: list[Camera],
#     camera_names: list[str],
#     *,
#     input_is_undistorted=True,
# ):

#     num_cameras = len(cameras)
#     if len(detections) == 0:
#         raise InvalidDetectionData("Detection is Empty")
#     num_frames = len(detections[0])

#     # check for missing info. This can happen when running intrinsic only in particular.
#     for c in cameras:
#         if c is None:
#             raise MissingCalibrationData("Calibration accuracy check is not possible.")
#         if c.extrinsic_matrix is None:
#             raise MissingCalibrationData(
#                 "Calibration accuracy check is not possible. Missing extrinsic matrices"
#             )
#         if c.intrinsic is None:
#             raise MissingCalibrationData(
#                 "Calibration accuracy check is not possible. Missing intrinsic matrices"
#             )
#         if not input_is_undistorted and c.distortion_coeff is None:
#             raise MissingCalibrationData(
#                 "Calibration accuracy check on distorted detections is not possible. Missing distortion coefficients."
#             )

#     print(f"Check accuracy for {num_cameras} cameras with {num_frames} detections.")

#     proj_matrices = get_projection_matrices(cameras)

#     # Accumulate per-camera reprojection errors
#     # camera_errors = defaultdict(list)

#     camera_errors_pixel = defaultdict(list)
#     camera_errors_mm = defaultdict(list)

#     triangulated_points = []

#     for frame in range(num_frames):
#         # print(f"{frame}/{num_frames}")
#         image_points_per_cam = []
#         detected_ids_per_cam = []

#         for cam_idx in range(num_cameras):
#             detection = detections[cam_idx][frame]
#             ids = detection["ids"]
#             corners = detection["corners"]

#             if ids is None or corners is None or len(ids) == 0:
#                 image_points_per_cam.append({})
#                 detected_ids_per_cam.append(set())
#                 continue

#             ids = ids.flatten()
#             corners = np.array(corners, dtype=np.float32).reshape(-1, 2)

#             # undistort
#             if not input_is_undistorted:
#                 corners = undistort_corners(corners, cameras[cam_idx])

#             point_map = {int(id_): corners[i] for i, id_ in enumerate(ids)}
#             image_points_per_cam.append(point_map)
#             detected_ids_per_cam.append(set(point_map.keys()))

#         # Find all marker corner IDs that are seen by at least 2 cameras
#         all_ids_flat = [id_ for ids in detected_ids_per_cam for id_ in ids]
#         if len(all_ids_flat) == 0:
#             triangulated_points.append([])
#             continue
#         id_counts = Counter(all_ids_flat)
#         common_ids = {id_ for id_, count in id_counts.items() if count >= 2}

#         trangulated_poins_per_frame = []

#         for point_id in common_ids:
#             views = []
#             Ps = []

#             for cam_idx in range(num_cameras):
#                 if point_id in image_points_per_cam[cam_idx]:
#                     views.append(image_points_per_cam[cam_idx][point_id])
#                     Ps.append(proj_matrices[cam_idx])

#             if len(views) < 2:
#                 continue

#             # Triangulate 3D point in world coords
#             pt_3d = triangulate_n_views(
#                 views, Ps
#             )  # [x,y,z] one location set per point on the board

#             trangulated_poins_per_frame.append(
#                 pt_3d
#             )  # we get a list of locaitons sets for this frame

#             for cam_idx in range(num_cameras):
#                 if point_id in image_points_per_cam[cam_idx]:
#                     obs = image_points_per_cam[cam_idx][point_id]
#                     P = proj_matrices[cam_idx]

#                     # Reproject triangulated point to 2D pixel
#                     reproj = reproject(P, pt_3d)
#                     pixel_error = np.linalg.norm(reproj - obs)

#                     camera_errors_pixel[camera_names[cam_idx]].append(pixel_error)

#                     # also get mm error
#                     # Compute depth of point in this camera's coords
#                     cam = cameras[cam_idx]
#                     R = cam.extrinsic_matrix[:3, :3]
#                     t = cam.extrinsic_matrix[:3, 3]
#                     pt_cam = R @ pt_3d + t
#                     depth_mm = pt_cam[2] * 1000.0  # assuming pt_3d in meters
#                     # Focal length in pixels (assume fx)
#                     fx = cam.intrinsic[0, 0]
#                     # Convert pixel reprojection error to mm
#                     error_mm = (pixel_error * depth_mm) / fx

#                     camera_errors_mm[camera_names[cam_idx]].append(error_mm)
#         triangulated_points.append(trangulated_poins_per_frame)

#     return triangulated_points, camera_errors_pixel, camera_errors_mm


# def print_errors(camera_errors_mm, camera_errors_pixel, camera_names):
#     # Print per-camera reprojection error statistics in mm
#     print("\nReprojection error per camera:")
#     for cam_id in camera_names:
#         errors_mm = camera_errors_mm.get(cam_id, [])
#         errors_pixel = camera_errors_pixel.get(cam_id, [])
#         if errors_mm:
#             print(
#                 f"{cam_id:<15} | Mean: {np.mean(errors_pixel):.2f} pixel ({np.mean(errors_mm):.2f} mm) | Std: {np.std(errors_pixel):.2f} pixel | Max: {np.max(errors_pixel):.2f} pixel | Points: {len(errors_pixel)}"
#             )
#         else:
#             print(f"{cam_id:<15} | No valid points")


# def plot_camera_errors(
#     camera_errors_mm: dict[str, list[float]],
#     camera_names: list[str],
#     output_path="ReprojectionErrors.png",
#     unit="pixel",
# ):
#     data = [camera_errors_mm.get(cam_id, []) for cam_id in camera_names]

#     plt.figure(figsize=(18, 6))
#     plt.boxplot(data, labels=camera_names, showmeans=True, showfliers=True)

#     plt.xticks(rotation=90)
#     plt.ylabel(f"Reprojection Error ({unit})")
#     plt.title("Per-Camera Reprojection Error Distribution")
#     plt.grid(True, linestyle="--", alpha=0.4)
#     plt.tight_layout()
#     plt.savefig(output_path)
#     # plt.show()


# def plot_3d_points_and_cameras(
#     triangulated_points,
#     cameras,
#     camera_names,
#     origin_capture_idx=None,
#     *,
#     save_path="BoardsAndCameras.png",
#     interactive=False,
# ):
#     # trianglualted points are a list[3dpoints per frame]

#     fig = plt.figure(figsize=(10, 8))
#     ax = fig.add_subplot(111, projection="3d")

#     # Plot triangulated points
#     for frame in range(len(triangulated_points)):
#         pts = np.array(triangulated_points[frame])
#         if len(pts) > 0:
#             ax.scatter(
#                 pts[:, 0],
#                 pts[:, 1],
#                 pts[:, 2],
#                 c="gray",
#                 marker="o",
#                 s=4,
#             )
#             ax.text(
#                 pts[0, 0],
#                 pts[0, 1],
#                 pts[0, 2],
#                 frame,
#                 color="gray",
#             )

#     if origin_capture_idx is not None and origin_capture_idx != -1:
#         origin_pts = np.array(triangulated_points[origin_capture_idx])
#         ax.scatter(
#             origin_pts[:, 0],
#             origin_pts[:, 1],
#             origin_pts[:, 2],
#             c="red",
#             s=4,
#             label=f"Origin Board (frame idx: {origin_capture_idx})",
#         )
#         ax.text(
#             origin_pts[0, 0],
#             origin_pts[0, 1],
#             origin_pts[0, 2],
#             origin_capture_idx,
#             color="red",
#         )

#     # Plot camera centers
#     for cam, cam_id in zip(cameras, camera_names):
#         R = cam.extrinsic_matrix[:3, :3]
#         t = cam.extrinsic_matrix[:3, 3]
#         cam_pos = -R.T @ t
#         color = "gray"
#         if cam_id in ["C0031", "C0039"]:
#             color = "cyan"
#         elif cam_id in ["C0018", "C0004", "C0034"]:
#             color = "magenta"
#         elif cam_id in ["C1002", "C0020"]:
#             color = "black"
#         ax.scatter(
#             cam_pos[0],
#             cam_pos[1],
#             cam_pos[2],
#             c="red",
#             marker="^",
#             s=50,
#         )
#         ax.text(cam_pos[0], cam_pos[1], cam_pos[2], cam_id, color=color)

#     ax.set_xlabel("X (m)")
#     ax.set_ylabel("Y (m)")
#     ax.set_zlabel("Z (m)")
#     ax.set_title("3D Triangulated Points and Cameras")
#     ax.legend()
#     if interactive:
#         plt.show()
#     print("Saved Plot - Boards and Cameras  --->", save_path)
#     plt.savefig(save_path)


In [ ]:
path = "/data/vci/motion_markers/2026_01_29_motion_markers_001"